# Финский язык

Фи́нский язы́к (фин. suomi, suomen kieli) — язык финнов, относящийся к прибалтийско-финской подгруппе финно-волжской группы финно-угорских языков; является агглютинативным языком номинативного строя со значительными элементами флективности[3]. Финно-угорские языки и самодийские языки (энецкий, ненецкий, нганасанский, селькупский) составляют уральскую языковую семью. Финский — один из 24 официальных языков Европейского союза.

## Постановка задачи

Предлагается реализовать алгоритм Витерби для создания PoS-тэггера.

На языке Python/R необходимо разработать PoS-теггер на базе скрытых марковских цепей и алгоритма Витерби.

In [37]:
import conllu  
from io import open
import pickle 
import numpy as np

In [38]:
direct = "./UD_Finnish/"
train = "fi_tdt-ud-train.conllu"
test  = "fi_tdt-ud-test.conllu"

In [39]:
data_train = conllu.parse_incr(open(direct+train,'r',encoding = "utf-8"))

In [40]:
c3 = {} 
c2 = {} 
c1 = {} 
cT = {} 
for tokenlist in data_train:
    sentence = [["*","*"]] + [["*","*"]] + [[tokenlist[i]["lemma"], tokenlist[i]["upostag"]]
        for i in range(len(tokenlist)) ] + [["STOP","STOP"]]

    for i in range(0, len(sentence)-2):
        c3old = c3.get( (sentence[i][1], sentence[i+1][1], sentence[i+2][1]), 0)
        c3.update( { (sentence[i][1], sentence[i+1][1], sentence[i+2][1]): c3old+1 } ) 
    for i in range(0, len(sentence)-1):
        c2old = c2.get( (sentence[i][1], sentence[i+1][1]), 0)
        c2.update( { (sentence[i][1], sentence[i+1][1]): c2old+1 } )

    for i in range(0, len(sentence)):
        c1old = c1.get( (sentence[i][1]), 0)
        c1.update( { (sentence[i][1]): c1old+1 } )

    for i in range(0, len(sentence)):
        cTold = cT.get( (sentence[i][1], sentence[i][0]), 0)
        cT.update( { (sentence[i][1], sentence[i][0]): cTold+1 } )

In [41]:
print (len(c3))
print (len(c2))
print (len(c1))
print (len(cT))

with open('cs_finnish.pkl', 'wb') as f:
    pickle.dump([cT, c1, c2, c3], f)

2086
235
17
23486


In [42]:
with open('cs_finnish.pkl', 'rb') as f:
    cT, c1, c2, c3 = pickle.load(f)

In [43]:
tags = list(c1.keys())
tags.remove('*')
tags.remove('STOP')
k = len(tags)

In [44]:
def q(s, u, v):
    eps = 1e-7
    return c3.get((u, v, s), eps) / c2.get((u, v), eps)


def e(x, s):
    eps = 1e-7
    return cT.get((s, x), eps) / c1.get((s), eps)


def K(n): 
    if n == -1 or n == 0:
        return ['*']
    else:
        return tags

print(K(10))

['NOUN', 'ADJ', 'VERB', 'ADV', 'CCONJ', 'PUNCT', 'SCONJ', 'PRON', 'PROPN', 'AUX', 'ADP', 'NUM', 'X', 'SYM', 'INTJ']


In [45]:
def viterbi(sent):
    Pi = {(0, '*', '*'): 1}
    bp = {}
    n = len(sent) - 2
    y = [""] * (n + 1)

    for k in range(1, n + 1):
        xk = sent[k]
       
        for u in K(k-1):
            for v in K(k):
                w = K(k-2)
                v1 = map(lambda wi:
                            Pi.get((k-1, wi, u)) *
                            q(v, wi, u) *
                            e(xk, v), w)
                v1 = list(v1)
                PiNew = max(v1)
                bpNew = w[v1.index(PiNew)]
                Pi.update({(k, u, v): PiNew})
                bp.update({(k, u, v): bpNew})

    v2 = {}
    for u in K(n-1):
        for v in K(n):
            v2.update({(n, u, v): Pi.get((n, u, v)) * q("STOP", u, v)})

    v2max = max(list(v2.values()))
    for (n, u, v), val in v2.items():
        if val == v2max:
            (y[n-1], y[n]) = (u,v)

    for k in range(n-2, 0, -1):
        y[k] = bp.get((k + 2, y[k+1], y[k+2]))

    return y[1:]

In [46]:
data_test = conllu.parse_incr(open(direct+test,'r',encoding = "utf-8"))
test_tags = [] 
predict_tags = [] 

for tokenlist in data_test:
    sentence=["*"] + [tokenlist[i]["lemma"] for i in range(len(tokenlist)) ] + ["STOP"]
    test_tags.append([tokenlist[i]["upostag"] for i in range(len(tokenlist)) ])
    predict_tags.append(viterbi(sentence))

In [47]:
err = 0.
for sent in range(len(test_tags)):
    err += np.mean(np.array(test_tags[sent]) != np.array(predict_tags[sent]))
print('Error = ', err/len(test_tags))

Error =  0.1133836397585961
